Устанавливаем библиотеки

In [ ]:
import warnings
warnings.filterwarnings('ignore')
!pip install -q numpy==1.24.4
import os
os.kill(os.getpid(), 9)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 32.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.0 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.


In [1]:
!pip install -q tensorflow==2.12
!pip install -q git+https://github.com/dvolchek/rnnmorph.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 8.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2025.1.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
bigframes 1.42.0 requires nump

Создаём объект морфологического анализатора `RNNMorph`

In [2]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

**Ссылка**, на источник текста

In [3]:
DATA_URL = "http://az.lib.ru/l/leskow_n_s/text_0246.shtml"

In [4]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [5]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Лесков Николай Семенович. Левша</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/l/leskow_n_s/>Лесков Николай Семенович</a><br>\r\nЛевша</h2>\r\n\r\n<!-------'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [7]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Лесков Николай Семенович. Левша\n\n\n\nЛесков Николай Семенович\r\nЛевша\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n[Новинки]\r\n[Обзоры]\r\n[Помощь]\r\n\r\n'

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


"A total of 466 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [9]:
from tqdm.notebook import tqdm
predictions = [[pred.normal_form for pred in sent]
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10]

8/8 [==============================] - 6s 546ms/step


sentences:   0%|          | 0/466 [00:00<?, ?it/s]

[['глава',
  'двадцатый',
  'теперь',
  'весь',
  'это',
  'уже',
  '``',
  'дело',
  'минувший',
  'день',
  "''",
  ':',
  'и',
  '``',
  'предание',
  'старина',
  "''",
  ',',
  'хотя',
  'и',
  'не',
  'глубокий',
  ',',
  'но',
  'предание',
  'этот',
  'нет',
  'нужда',
  'торопиться',
  'забывать',
  ',',
  'несмотря',
  'на',
  'баснословный',
  'склад',
  'легенда',
  'и',
  'эпический',
  'характер',
  'он',
  'главный',
  'герой',
  '.']]

In [10]:
predictions = [[token for token in sent if token.isalpha()] for sent in predictions]

In [11]:
len(predictions)

466

In [12]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens)


9053

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 200 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам.

In [13]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['и', 'в', 'во', 'не', 'что']

In [16]:
# получим 200 самых частотных токенов
fdist = FreqDist(word.lower() for word in non_uniq_tokens)
most_common_200 = fdist.most_common(200)

# получим токены, не относящиеся к стоп-словам
non_stopword_tokens = [token for token, frequency in most_common_200 if token not in STOPWORDS]

# рассчитаем долю
non_stopword_tokens_fraction = len(non_stopword_tokens) / 200
print(f'Доля токенов, не относящихся к стоп словам среди 200 самых частотных токенов: {non_stopword_tokens_fraction:.2f}')

Доля токенов, не относящихся к стоп словам среди 200 самых частотных токенов: 0.59


## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 100 раз.

In [17]:
tokens_above_100 = [token for token, frequency in fdist.items() if frequency > 100]
tokens_above_100_count = len(tokens_above_100)
print(f'Кол-во токенов, встречающихся в тексте строго больше 100 раз: {tokens_above_100_count}')

Кол-во токенов, встречающихся в тексте строго больше 100 раз: 8
